In [1]:
import pymol

import mbuild as mb
from mbuild.lib.recipes.polymer import Polymer

import warnings
warnings.filterwarnings('ignore')

import mdtraj as md
from openff.interchange import Interchange
from openmm import app
from openff.toolkit import ForceField, Molecule
from openff.toolkit.utils import get_data_file_path

# Build PDB of helical peptide

## PyMol Scripts
First we build the peptide sequence using PyMOLs built in peptide builder. Here we use the scripting version of PyMOL to build this peptide without interfacing with the PyMOL GUI.

In [2]:
peptide_sequence = "EEEEKKKK"
pymol.cmd.delete("all")
pymol.editor.fab(peptide_sequence, ss = 1)
pymol.cmd.set("pdb_conect_all", "on")
pymol.cmd.save("short_peptide.pdb")

Next we'll use mbuild to add our capping residues to the generated PyMol generated peptide pdb.

In [3]:
# Use mbuild to visualize componenets
peptide = mb.load("short_peptide.pdb")
peptide
view = peptide.visualize(show_ports=True)
style = {
                "stick": {"radius": 0.2, "color": "grey"},
                "sphere": {"scale": 0.6, "color" : "black"},
    }
view.setStyle({'model': -1, 'serial':1},style)
view.setStyle({'model': -1, 'serial':129},style)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [4]:
succynl = mb.load("O=C(O)CCC(=O)O", smiles=True)
succynl.save("succ.pdb", overwrite=True)
succynl.remove(succynl[8])
view = succynl.visualize()

style = {
                "stick": {"radius": 0.2, "color": "grey"},
                "sphere": {"scale": 0.3, "color" : "black"},
    }
view.setStyle({'model': -1, 'serial':3},style)
#succynl = mb.load("O=C([O-])CCC(=O)O", smiles=True)
#view = succynl.visualize()
view.setStyle({'model': -1, 'serial':3},style)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [5]:
nh3 = mb.load("N", smiles=True)
nh3.save("nh3.pdb", overwrite=True)
view = nh3.visualize()
style = {
                "stick": {"radius": 0.2, "color": "grey"},
                "sphere": {"scale": 0.6, "color" : "black"},
    }
view.setStyle({'model': -1, 'serial':2},style)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [6]:
h2o = mb.load("O", smiles=True)
h2o.save("h2o.pdb", overwrite=True)
h2o.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Now that we have our peptide and cap residues, we will use `mbuild` Polymer object to combine them.

In [7]:
chain = Polymer()

chain.add_monomer(compound = peptide, indices=[0, 128], separation=0.15, replace = False)
chain.add_end_groups(compound = succynl,
                     index = 2,
                     separation = 0.15,
                     label = "head",
                     duplicate = False,
                     orientation=[0.5,-0.5,0.5]
                    )

chain.add_end_groups(compound = nh3,
                     index = 2,
                     separation = 0.15,
                     label = "tail",
                     duplicate = False
                    )

chain.build(n=1)

In [8]:
peptide = chain.children[0]

We have to do some reassignment of bonds. I think the bonds list altered here, so mbuild thikns the aromatic hydrogens are not attached to the ring

In [9]:
# tyr = peptide.children[0]
# for bond in tyr.bonds():
#     print(bond[0].name, bond[1].name)

In [10]:
# for i, atom in enumerate(tyr.children):
#     if atom.name[0].isdigit():
#         print("Switching", atom.name, "to", atom.name[1:] + atom.name[0])
#         atom.name = atom.name[1:] + atom.name[0]
#     print(atom.name)
        

In [11]:
# manually adding lost tyrosine bonds
# tyr.add_bond([tyr[6], tyr[16]])
# tyr.add_bond([tyr[7], tyr[17]])
# tyr.add_bond([tyr[8], tyr[18]])
# tyr.add_bond([tyr[9], tyr[19]])

In [12]:
# for b in chain.children[1].bonds():
#     print(b[0].name, b[1].name)

In [13]:
chain.energy_minimize()
chain.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [14]:
# Set labels for capping residues
print(chain.children[1])
chain.children[1].name = "NTC"

print(chain.children[2])
chain.children[2].name = "CTC"

<Compound 12 particles, 11 bonds, non-periodic, id: 140290409442816>
<Compound 3 particles, 2 bonds, non-periodic, id: 140290409445552>


In [15]:
# Verify we relabeled the non-standard capping residues
chain.labels['Compound'][0]

<NTC 12 particles, 11 bonds, non-periodic, id: 140290409442816>

Now we save the peptide to file, ensuring amino acid residue names are written correctly. We also add the residues `NTC` and `CTC` for the N-terminal and C-terminal caps.

In [16]:
chain.save("mbuild_small_peptide.pdb", overwrite = True, residues=["ALA", "LYS", "GLU", "TYR", "SER", "NTC", "CTC"])
smiles = chain.to_smiles()